# Group Project: TorchText를 이용한 Naver Movie Review 분류 구현

## Specs
- https://github.com/bentrevett/pytorch-sentiment-analysis 는 Pytorch, TorchText를 이용하여 IMDB 분류를 다양한 방법으로 행하고 있는 사이트이다.
- 이 사이트를 참고로 하여, Naver News Review를 TorchText를 이용하여 분류하는 프로그램을 구현하라.
- 3명 이내의 그룹이나 혼자해도 무방!
- 여러 가능한 딥러닝 모듈을 사용할 수 있고(Transformer는 제외), 같이 제공되는 이 영화평으로 학습된 word2vec 임베딩을 반드시 포함하여야 한다.
- TorchText는 제공되는 이외의 임베딩은 따로 구현해야 하기 때문에 한국어 임베딩을 TorchTExt에 통합시키는 방법을 모색해 보라.
- 그룹으로 하는 경우 파일 이름 FinalProjectYOURGroupName.ipynb로 하고 요약에 참가자들 학번과 명단을 명시해야 함. 그룹명은 자유롭게 할 수 있으나 너무 길지 않도록
- 혼자하는 경우 그룹명에는 학번이 들어가야함(FinalProject2019_01234.ipynb)
- 아래 셀은 가이드로 제시된 것이니 필요에 따라 수정할 수 있음
- 마감: 6월 4일 수업시간 전까지
- 참고 사이트
    - https://pytorch.org/text/
    - http://mlexplained.com/2018/02/08/a-comprehensive-tutorial-to-torchtext/
    - https://github.com/pytorch/text 
    - https://mc.ai/using-fine-tuned-gensim-word2vec-embeddings-with-torchtext-and-pytorch/
    - 조교가 올린 newsdata 모델

# __Team "WeCute"__
##### 2016-12670 김종호 
##### 2016-15726 정수민  

----------------------------------------


## 한국어 뉴스 데이터 전처리(tokenizer)
- 한자 제거: hanja 모듈을 사용해서 한자어를 한글로 바꾼다. 그런데 hanja를 사용해도 vocab의 수가 동일한 것으로 보아 한자어로 제시된 단어들은 이미 다른 문장에서 한글로 적힌 데이터를 통해서 vocab에 포함되었다고 볼 수 있다.
- 명사 추출: 의미론적 결정력이 있는 명사만 추출한다. 명사 추출은 konlpy에서 제공하는 Okt 의 tag를 사용하였다.
    - Komoran 과 Okt의 비교
    - Komoran은 복합어를 그대로 놔두는 반면, Okt는 복합어를 쪼갠다는 점에서 차이가 있다. 둘 중에서 Okt를 선택한 이유는 다음과 같다. 첫째, Komoran은 Okt보다 시간이 훨씬 오래 걸리기 때문에 비효율적이다. 둘째,  뉴스 데이터를 분류할 때 복합어를 굳이 쪼개지 않더라도 결정력을 가질 것이다. 가령 ‘조국평화통일위원회’의 경우 Okt에서 ‘조국', ‘평화통일’ ‘위원회'로 분석되는데, 이 단어들 또한 뉴스 데이터 분류에서 결정력을 가진다. 셋째, 아래에 서술하겠지만, Bigram을 사용하여 복합어가 가지는 의미적 결정력을 포괄할 수 있다. 따라서 Okt를 사용하는 것이 더 효율적이라고 판단하였다.
- 불용어 제거: 선행 연구(길호현, 2018)를 참고하여 Stopwords 목록을 만들었다.			
    - 길호현 (2018). 텍스트마이닝을 위한 한국어 불용어 목록 연구. 우리말글 , 78, 1-25 에서 제시된 불용어 목록을 사용했다. 명사에서 그것, 그곳, 그녀 등 대명사와 녀셕, 마당, 바퀴 등 의존명사를 제거하였다.

## Vocab 설정
-  news 데이터에서의 토큰: Preprocessing = bigrams 옵션을 주어 “대한민국 킥복싱이 아시아 40여개국…” 에서 “대한민국 킥복싱" “킥복싱이 아시아" 와 같이 bi- gram 단어 또한 분석에 포함하였다.
-  외부 임베딩: “NewsAllW2vec.txt” 에 나와 있는 word2vec 을 import 하였다. 
- news 데이터와 주어진 외부 데이터에 나와 있는 단어들을 임베딩하여 모델의 input으로 사용하였다.

## 모델 설정
- 하이퍼 파라미터: batch_size = 64,  learning rate = 0.01, epoch = 50, optimizer는 Adam
- 이는 우선 수업 시간에 배운 내용과 몇 번의 시행착오를 기반으로 설정하였고, 이를 적용하여 모델을 정한 뒤에 조정해볼 것이다. 
- sequence data를 효과적으로 인식하는 RNN,
- RNN의 보완 버전으로 이전 데이터를 기록해 sequence를 포함하여 학습하는 LSTM, 
- 필터를 활용하여 3차원의 데이터를 효과적으로 분석하는 CNN, 
- 그리고 n-grams를 활용하는 것이 큰 특징인 FastText를 만들고 각각의 모델에 데이터를 학습시켰다. 
- 결과적으로 test set의 accuracy가 가장 좋게 나온 FastText를 채택하였다. 분석에 사용한 각각의 코드는 아래 class로 지정해두었다. 다만 모델에 따라 field에 지정해야 하는 파라미터가 다른데, 이는 https://github.com/bentrevett/pytorch-sentiment-analysis 를 참고해서 적용시켰다. 본 파일은 FastText에 맞게끔 설정되어 있다.
- 앞으로 우리 모델과 비교할 대안 모델을 돌렸을 때 나오는 해당 epoch 내 test accuracy 최대값을 __진하게__ 표시할 것이다.


#### 1) RNN

Epoch: 50 | Epoch Time: 0m 4s  
	Train Loss: 0.434 | Train Acc: 89.14%  
	Test. Loss: 4.311 | Test. Acc: __12.19%__


#### 2) LSTM

Epoch: 50 | Epoch Time: 0m 4s  
	Train Loss: 0.003 | Train Acc: 100.00%  
	Test. Loss: 1.068 |  Test. Acc: __80.00%__

#### 3) CNN 

Epoch: 50 | Epoch Time: 0m 0s  
	Train Loss: 0.011 | Train Acc: 99.92%  
	Test. Loss: 0.645 |  Test. Acc: __80.94%__

#### 4) FastText  
- 이 모델로 선택했기 때문에 전체적인 Epoch 경향 제시

Epoch: 01 | Epoch Time: 0m 0s  
	Train Loss: 1.872 | Train Acc: 40.47%  
	Test Loss: 1.705 |  Test Acc: 66.25%

Epoch: 02 | Epoch Time: 0m 0s  
	Train Loss: 1.305 | Train Acc: 75.55%  
	Test Loss: 1.253 |  Test Acc: 72.19%

Epoch: 03 | Epoch Time: 0m 0s  
	Train Loss: 0.749 | Train Acc: 87.03%  
	Test Loss: 0.917 |  Test Acc: 79.38%

Epoch: 04 | Epoch Time: 0m 0s  
	Train Loss: 0.415 | Train Acc: 92.34%  
	Test Loss: 0.743 |  Test Acc: 80.94%

(생략)

Epoch: 28 | Epoch Time: 0m 0s  
	Train Loss: 0.002 | Train Acc: 100.00%  
	Test Loss: 0.525 |  Test Acc: __86.88%__

(생락)

Epoch: 50 | Epoch Time: 0m 0s  
	Train Loss: 0.001 | Train Acc: 100.00%  
	Test Loss: 0.536 |  Test Acc: 86.56%



## Optimizer 설정
- Optimizer는 backpropagation을 효율적으로 하기 위해 gradient descent를 구하는 방식과 learning rate을 조절한다
- 그 중에서 성능이 좋다고 알려진 Momentum, Adam, Adagrad, RMSprop을 각각 실시해보았다. 
- 이를 통해 가장 성능이 좋은 Adam을 채택하였다. 
- 여기에 Scheduler를 해보았으나 test accuracy가 오히려 감소하여 넣지 않았다. 

#### 1) SGD_Momentum

Epoch: 50 | Epoch Time: 0m 0s  
	Train Loss: 1.291 | Train Acc: 64.14%  
	Test. Loss: 1.434 | Test. Acc: __58.75%__

#### 2) Adam

Epoch: 50 | Epoch Time: 0m 0s  
	Train Loss: 0.001 | Train Acc: 100.00%  
	Test. Loss: 0.535 |  Test. Acc: __86.56%__

#### 3) Adagrad

Epoch: 50 | Epoch Time: 0m 0s  
	Train Loss: 0.293 | Train Acc: 95.78%  
	Test Loss: 0.756 |  Test Acc: __79.69%__

#### 4) RMSprop

Epoch: 50 | Epoch Time: 0m 0s  
	Train Loss: 0.063 | Train Acc: 99.38%  
	Test Loss: 0.570 |  Test Acc: __82.81%__

#### 5) Adagrad에 Scheduler 적용

_scheduler = StepLR(optimizer, step_size=1, gamma=0.1)_  

Epoch: 50 | Epoch Time: 0m 0s  
	Train Loss: 0.000 | Train Acc: 100.00%  
	Test Loss: 0.867 |  Test Acc: __84.69%__ 
 
## overfitting 해결 방법
- 우리의 FastText 모델의 Train accuracy는 100%로, train set에 overfitting 되어 test set을 잘 예측하지 못할 것이라는 우려가 있을 수 있다.
- overfitting을 해결하기 위해 알려진 방법으로는 training set의 표본 늘리기, weight decay, dropout, early stopping 등이 있다. 이 중에서 표본은 충분하다고 판단하여, 나머지 세 개를 모델에 각각 적용해보았다.


#### 1) weight decay
- 오히려 test accuracy가 감소하여 추가하지 않았다.

_optimizer = optim.Adam(model.parameters(), lr=0.01, weight_decay=1e-5)_

Epoch: 50 | Epoch Time: 0m 0s  
	Train Loss: 0.012 | Train Acc: 100.00%  
	Test. Loss: 0.607 |  Test. Acc: __83.12%__

#### 2) dropout
- FastText model에 dropout=0.5로 넣고 뉴럴넷을 일부 지워보았지만 test accuracy 증가는 역시 나타나지 않아서 추가하지 않았다.

#### 3) early stopping
- 앞서 제시한 것처럼 Epoch가 50인 경우 overfitting이 일어나서 Test Accuracy 가 86.88%에서 86.56%으로 오히려 감소했다. 이에 Test Acc가 최대가 되게끔 하기 위해 30으로 조정하였다.

## Hyper-parameter 조정
- 마지막으로 성능을 더 높이기 위하여 하이퍼파라미터를 조정해보았다.

#### 1) learning rate = 0.01
- 0.001

Epoch: 30 | Epoch Time: 0m 0s  
	Train Loss: 0.429 | Train Acc: 94.06%  
	 Test Loss: 0.877 |  Test Acc: 79.38%

Epoch: 100 | Epoch Time: 0m 0s  
	Train Loss: 0.019 | Train Acc: 100.00%  
	 Test Loss: 0.543 |  Test Acc: __85.62%__

- 0.1  

Epoch: 30 | Epoch Time: 0m 0s  
	Train Loss: 0.000 | Train Acc: 100.00%  
	 Test Loss: 0.649 |  Test Acc: 85.00%

Epoch: 50 | Epoch Time: 0m 0s  
	Train Loss: 0.000 | Train Acc: 100.00%  
	 Test Loss: 0.665 |  Test Acc: __85.31%__

#### 2) batch size = 64
- 32

Epoch: 30 | Epoch Time: 0m 0s  
	Train Loss: 0.001 | Train Acc: 100.00%  
	 Test Loss: 0.531 |  Test Acc: __86.25%__

- 128

Epoch: 21 | Epoch Time: 0m 0s  
	Train Loss: 0.012 | Train Acc: 100.00%  
	 Test Loss: 0.542 |  Test Acc: __86.46%__

Epoch: 30 | Epoch Time: 0m 0s  
	Train Loss: 0.005 | Train Acc: 100.00%  
	 Test Loss: 0.531 |  Test Acc: 85.94%

## 결론

- FastText Model (with bigrams)
- Hyper-parameter
	- batch size = 64  
	- learning rate = 0.01
	- epoch = 30
	- optimizer: Adam
- train set 정확도: 100%, test set의 정확도: 86.88%

In [283]:
!pip install konlpy

In [284]:
!pip install hanja

Processing /root/.cache/pip/wheels/68/58/66/f1fc9afad272df0df0fa38cdef34ee2d5d8d6a85f4eb5acab5/coverage-3.7.1-cp36-cp36m-linux_x86_64.whl
ERROR: pytest-cov 2.9.0 has requirement coverage>=4.4, but you'll have coverage 3.7.1 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
  Found existing installation: coverage 5.1
    Uninstalling coverage-5.1:
      Successfully uninstalled coverage-5.1


## 구현 요약
- 각 조가 구현한 내용, 특징, 성능, 결과 등을 종합적으로 정리.
- 그림 파일을 포함할 경우 별도의 그림 데이터없이 이 노트북에서 곧장 열려야 함

In [0]:
## 필요한 모듈
import numpy as np
import pandas as pd
import torch
import torch.utils.data
import konlpy
from konlpy.tag import Twitter
from konlpy.tag import Okt
import hanja
from torchtext import data

## 데이터 
- 제공되는 데이터는 Naver News 데이터의 헤드라인과 본문을 합친 news 컬럼이 있는 데이터
![%E1%84%89%E1%85%B3%E1%84%8F%E1%85%B3%E1%84%85%E1%85%B5%E1%86%AB%E1%84%89%E1%85%A3%E1%86%BA%202020-05-17%20%E1%84%8B%E1%85%A9%E1%84%8C%E1%85%A5%E1%86%AB%2011.45.36.png](attachment:%E1%84%89%E1%85%B3%E1%84%8F%E1%85%B3%E1%84%85%E1%85%B5%E1%86%AB%E1%84%89%E1%85%A3%E1%86%BA%202020-05-17%20%E1%84%8B%E1%85%A9%E1%84%8C%E1%85%A5%E1%86%AB%2011.45.36.png)

## Tokenizer 구현
- TorchText에서는 기본적으로 string 단위로 토크나이즈 하고, spacy와 같은 외부 토크나이저를 사용할 수 있음
- 한국어의 경우는 형태소 분석을 해야하기 때문에 형태소 분석을 하는 모듈을 FIELD에 포함시켜야 함
- TEXT의 preprocessing으로 연결할 수 있는 형태소 분석 모듈 작성(조교의 이전 프로그램 참조)

In [0]:
# 불용어 처리
str_stopwords="가량	 가지	 각	 간	 갖은	 개	 개국	 개년	 개소	 개월	 걔	 거	 거기	 거리	 건	 것	 겨를	 격	 겸	 고	 군	 군데	 권	 그	 그거	 그것	 그곳	 그까짓	 그네	 그녀	 그놈	 그대	 그래	 그래도	 그서	 그러나	 그러니	 그러니까	 그러다가	 그러면	 그러면서	 그러므로	 그러자	 그런	 그런 데	 그럼	 그렇지만	 그루	 그리고	 그리하여	 그분	 그이	 그쪽	 근	 근데	 글쎄	 글쎄요	 기	 김	 나	 나름	 나위	 남짓	 내	 냥	 너	 너희	 네	 네놈	 녀석	 년	 년대	 년도	 놈	 누구	 니	 다른	 다만	 단	 달	 달러	 당신	 대	 대로	 더구나	 더욱이	 데	 도	 동	 되	 두	 두세	 두어	 둥	 듯	 듯이	 등	 등등	 등지	 따라서	 따름	 따위	 딴	 때문	 또	 또는	 또한	 리	 마당	 마련	 마리	 만	 만큼	 말	 매	 맨	 명	 몇	 몇몇	 모	 모금	 모든	 무렵	 무슨	 무엇	 뭐	 뭣	 미터	 및	 바	 바람	 바퀴	 박	 발	 발짝	 번	 벌	 법	 별	 본	 부	 분	 뻔	 뿐	 살	 새	 서너	 석	 설	 섬	 세	 세기	 셈	 쇤네	 수	 순	 스무	 승	 시	 시간	 식	 씨	 아	 아냐	 아니	 아니야	 아무	 아무개	 아무런	 아아	 아이	 아이고	 아이구	 야	 약	 양	 얘	 어	 어느	 어디	 어머	 언제	 에이	 엔	 여기	 여느	 여러	 여러분	 여보	 여보세요	 여지	 역시	 예	 옛	 오	 오랜	 오히려	 온	 온갖	 올	 왜냐하면	 왠	 외	 요	 우리	 원	 월	 웬	 위	 음	 응	 이	 이거	 이것	 이곳	 이놈	 이래	 이런	 이런저런	 이른바	 이리하여	 이쪽	 일	 일대	 임마	 자	 자기	 자네	 장	 저	 저것	 저기	 저놈	 저런	 저쪽	 저편	 저희	 적	 전	 점	 제	 조	 주	 주년	 주일	 줄	 중	 즈음	 즉	 지	 지경	 지난	 집	 짝	 쪽	 쯤	 차	 참	 채	 척	 첫	 체	 초	 총	 측	 치	 큰	 킬로미터	 타	 터	 턱	 톤	 통	 투	 판	 퍼센트	 편	 평	 푼	 하기야	 하긴	 하물며	 하지만	 한	 한두	 한편	 허허	 헌	 현	 호	 혹은	 회	 흥"
stopwords=str_stopwords.split('\t ')
what_to_append=['1월', '2월', '3월', '4월', '5월', '6월', '7월', '8월', '9월' ,'10월', '11월', '12월', '!!']
stopwords.extend(what_to_append)

def tokenizer(string):
    string.replace("…", "")
    string.replace("·", "")
    analyzer = Okt()
    string=hanja.translate(string, 'substitution')
    nouns = analyzer.nouns(string)          #명사만 추출
    nouns = [a for a in nouns if (len(a) >1 and a not in stopwords) ]
    return nouns

def generate_bigrams(x):
    n_grams = set(zip(*[x[i:] for i in range(2)]))
    for n_gram in n_grams:
        x.append(' '.join(n_gram))
    return x

In [0]:
TEXT = data.Field(sequential=True, tokenize=tokenizer, preprocessing = generate_bigrams)
LABEL = data.LabelField(use_vocab=False)

## Data Connecting to Raw Data 
- TorchTExt를 이용하여, Naver News Data를 불러오고, TEXT, LABEL로 연결
- 불러온 데이터를 train, test 데이터로 나누고(8:2비율) train 데이터로 vocab을 형성
- test 데이터를 포함해서 vocab을 형성하면 안됨!

In [0]:
#@title 기본 제목 텍스트 { display-mode: "code" }
#전체 데이터를 raw_data로 불러들이기

from torchtext.data import TabularDataset

raw_datafields = [("filename", None),("headline", None),("body", None), \
                  ("label", LABEL),("news", TEXT)]

raw_data = data.TabularDataset(path="newsdataCombined.tsv", format='tsv', \
                               skip_header=True, fields=raw_datafields)



In [289]:
# TO check imported raw Data
print(f'Number of All examples: {len(raw_data)}')
print(vars(raw_data.examples[0]))

Number of All examples: 1600
{'label': '1', 'news': ['감자', '가격', '고공', '행진', '정부', '밥상', '물가', '잡기', '안간힘', '평양', '냉면', '인기', '강세', '일조', '가격', '상품', '수입', '조기', '출하', '세종', '연합뉴스', '이태수', '기자', '감자', '일부', '농산물', '가격', '정부', '상승', '밥상', '물가', '대란', '우려', '진정', '위해', '정부', '농림축산식품부', '급등', '농산물', '조기', '출하', '골자', '주요', '품목', '수급', '상황', '전망', '향후', '대책', '주요', '농산물', '가격', '점차', '안정화', '농식품부', '최근', '농산물', '가격', '한파', '지난달', '일시', '저온', '일조', '부족', '작황', '하량', '평년', '시세', '보이', '감자', '평년', '대비', '시세', '양파', '마늘', '재배', '면적', '증가', '시세', '우려', '설명', '감자', '지난해', '작황', '부진', '저장', '물량', '한파', '생육', '급량', '부족', '도매', '가격', '기준', '불리', '기도', '농식품부', '최근', '시설', '감자', '하량', '수입', '물량', '추가', '가격', '하향', '추세', '상순', '도매', '가격', '기준', '평년', '거래', '가격', '상황', '설명', '정부', '감자', '공급', '부족', '노지', '감자', '출하', '이전', '수입량', '농협', '통한', '조기', '출하', '적극', '유도', '방침', '지금', '수입', '감자', '정부', '한파', '겨울', '장량', '평년', '급량', '상황', '농식품부', '보도자료', '통해', '급량', '절대', '상황', '최근', '평양', '냉면', '인기', '육수', '고명'

In [0]:
# raw_data를 80:20으로 train_data, test_data로 랜덤하게 분리
import random

SEED=1234
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic=True
train_data, test_data = raw_data.split(split_ratio=0.8, random_state=random.seed(SEED))

In [291]:
print(f'Number of training examples: {len(train_data)}')
print(f'Number of testing examples: {len(test_data)}')
print(vars(train_data[0]))

Number of training examples: 1280
Number of testing examples: 320
{'label': '2', 'news': ['스튜디오', '집단', '성추행', '피해자', '비공개', '조사', '성악', '성범죄', '사건', '종합', '성악', '성범죄', '집중', '단속', '사건', '내일', '피의자', '소환', '성추행', '합의', '입장', '팽팽', '경찰', '집중', '조사', '서울', '뉴스', '최동현', '기자', '경찰', '스튜디오', '집단', '성추행', '의혹', '사건', '성악', '성범죄', '집중', '단속', '추진', '계획', '사건', '삼고', '본격', '수사', '착수', '서울', '마포', '경찰서', '오후', '서울', '시내', '유튜버', '양예원', '배우', '지망', '이소윤', '비밀리', '비공개', '고소', '조사', '진행', '비공개', '조사', '언론', '신분', '고소', '거부', '의사', '경찰', '결정', '조사', '시각', '장소', '모두', '경찰', '조사', '내용', '검토', '닉네임', '토니', '로만', '피의자', '신분', '소환', '전체', '사건', '적용', '혐의', '입건', '대상자', '결정', '방침', '이번', '사건', '경찰청', '발표', '성악', '성범죄', '집중', '단속', '추진', '계획', '사건', '경찰', '이번', '사건', '마포', '여성', '청소년', '수사', '전담', '수사', '배당', '서울', '지방', '경찰청', '수사', '합동', '수사', '양씨', '이씨', '고소장', '접수', '경찰', '주장', '검토', '성폭력', '범죄', '특례법', '위반', '카메라', '이용', '촬영', '강제추행', '협박', '혐의', '잠정', '적용', '사람', '노출', '사진', '유포', '인터넷', '음란', '사이트'

## Vocab 구축

In [0]:
MAX_VOCAB_SIZE = 20_000

TEXT.build_vocab(train_data, max_size = MAX_VOCAB_SIZE)
LABEL.build_vocab(train_data)

In [293]:
# vocab 테스트 

print(f"Unique tokens in TEXT vocabulary: {len(TEXT.vocab)}")
print(f"Unique tokens in LABEL vocabulary: {len(LABEL.vocab)}")

Unique tokens in TEXT vocabulary: 20002
Unique tokens in LABEL vocabulary: 8


In [294]:
print(TEXT.vocab.freqs.most_common(20))

[('기자', 1036), ('한국', 944), ('서울', 892), ('북한', 693), ('대해', 668), ('미국', 644), ('대한', 625), ('대통령', 581), ('관련', 558), ('이번', 554), ('위해', 544), ('경찰', 536), ('사람', 531), ('중국', 526), ('통해', 525), ('정부', 510), ('이후', 497), ('대표', 483), ('사실', 463), ('문제', 455)]


In [295]:
print(TEXT.vocab.itos[:10])

['<unk>', '<pad>', '기자', '한국', '서울', '북한', '대해', '미국', '대한', '대통령']


In [296]:
print(LABEL.vocab.stoi)
print(TEXT.vocab.stoi)

defaultdict(<function _default_unk_index at 0x7fa85efdf7b8>, {'0': 0, '3': 1, '6': 2, '4': 3, '7': 4, '2': 5, '5': 6, '1': 7})
defaultdict(<function _default_unk_index at 0x7fa85efdf7b8>, {'<unk>': 0, '<pad>': 1, '기자': 2, '한국': 3, '서울': 4, '북한': 5, '대해': 6, '미국': 7, '대한': 8, '대통령': 9, '관련': 10, '이번': 11, '위해': 12, '경찰': 13, '사람': 14, '중국': 15, '통해': 16, '정부': 17, '이후': 18, '대표': 19, '사실': 20, '문제': 21, '조사': 22, '후보': 23, '지난해': 24, '혐의': 25, '의원': 26, '경우': 27, '대회': 28, '관계자': 29, '상황': 30, '라며': 31, '최근': 32, '주장': 33, '진행': 34, '시장': 35, '자신': 36, '지역': 37, '설명': 38, '결과': 39, '현재': 40, '확인': 41, '예정': 42, '이상': 43, '트럼프': 44, '뉴스': 45, '오후': 46, '사건': 47, '시작': 48, '모두': 49, '방송': 50, '당시': 51, '입장': 52, '경기': 53, '회담': 54, '가장': 55, '세계': 56, '회장': 57, '수사': 58, '정상회담': 59, '국내': 60, '연합뉴스': 61, '사용': 62, '드루': 63, '올해': 64, '정도': 65, '모습': 66, '생각': 67, '전자': 68, '가능성': 69, '공개': 70, '계획': 71, '가격': 72, '게임': 73, '민주당': 74, '발표': 75, '일본': 76, '발생': 77, '상태': 78, '여성': 79, '사업':

## Data Load for Training

In [0]:
BATCH_SIZE = 64

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, test_data), 
    batch_size = BATCH_SIZE,
    sort_key=lambda x: len(x.news),
    sort_within_batch=True,
    device = device)

## Model 구축
- 사용하고자 하는 모델 구축
- 참고자료에 있는 방법등을 기초로 하여 여러 방법을 시도한 후 가장 성능이 좋은 모델을 선정.
- 이 선정에 대한 자세한 것도 기술해 놓아야 함

In [0]:
'''RNN'''
import torch.nn as nn

class RNN(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim):
        
        super().__init__()
        
        self.embedding = nn.Embedding(input_dim, embedding_dim)
        
        self.rnn = nn.RNN(embedding_dim, hidden_dim)
        
        self.fc = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, text):

        #text = [sent len, batch size]
        
        embedded = self.embedding(text)
        
        #embedded = [sent len, batch size, emb dim]
        
        output, hidden = self.rnn(embedded)
        
        #output = [sent len, batch size, hid dim]
        #hidden = [1, batch size, hid dim]
        
        assert torch.equal(output[-1,:,:], hidden.squeeze(0))
        
        return self.fc(hidden.squeeze(0))

In [0]:
"""LSTM, multi-layer, bidirectinoal LSTM"""
import torch.nn as nn

class RNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, 
                 bidirectional, dropout, pad_idx):
        
        super().__init__()
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx = pad_idx)
        
        self.rnn = nn.LSTM(embedding_dim, 
                           hidden_dim, 
                           num_layers=n_layers, 
                           bidirectional=bidirectional, 
                           dropout=dropout)
        
        self.fc = nn.Linear(hidden_dim * 2, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text, text_lengths):
        
        #text = [sent len, batch size]
        
        embedded = self.dropout(self.embedding(text))
        
        #embedded = [sent len, batch size, emb dim]
        
        #pack sequence
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths)
        
        packed_output, (hidden, cell) = self.rnn(packed_embedded)
        
        #unpack sequence
        output, output_lengths = nn.utils.rnn.pad_packed_sequence(packed_output)

        #output = [sent len, batch size, hid dim * num directions]
        #output over padding tokens are zero tensors
        
        #hidden = [num layers * num directions, batch size, hid dim]
        #cell = [num layers * num directions, batch size, hid dim]
        
        #concat the final forward (hidden[-2,:,:]) and backward (hidden[-1,:,:]) hidden layers
        #and apply dropout
        
        hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1))
                
        #hidden = [batch size, hid dim * num directions]
            
        return self.fc(hidden)
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
HIDDEN_DIM = 256
OUTPUT_DIM = 8
N_LAYERS = 2
BIDIRECTIONAL = True
DROPOUT = 0.5
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

model = RNN(INPUT_DIM, 
            EMBEDDING_DIM, 
            HIDDEN_DIM, 
            OUTPUT_DIM, 
            N_LAYERS, 
            BIDIRECTIONAL, 
            DROPOUT, 
            PAD_IDX)

In [0]:
'''CNN'''

import torch.nn as nn
import torch.nn.functional as F
class CNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, n_filters, filter_sizes, output_dim, 
                 dropout, pad_idx):
        
        super().__init__()
                
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx = pad_idx)
        
        self.convs = nn.ModuleList([
                                    nn.Conv2d(in_channels = 1, 
                                              out_channels = n_filters, 
                                              kernel_size = (fs, embedding_dim)) 
                                    for fs in filter_sizes
                                    ])
        
        self.fc = nn.Linear(len(filter_sizes) * n_filters, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text):
                
        #text = [batch size, sent len]
        
        embedded = self.embedding(text)
                
        #embedded = [batch size, sent len, emb dim]
        
        embedded = embedded.unsqueeze(1)
        
        #embedded = [batch size, 1, sent len, emb dim]
        
        conved = [F.relu(conv(embedded)).squeeze(3) for conv in self.convs]
            
        #conved_n = [batch size, n_filters, sent len - filter_sizes[n] + 1]
                
        pooled = [F.max_pool1d(conv, conv.shape[2]).squeeze(2) for conv in conved]
        
        #pooled_n = [batch size, n_filters]
        
        cat = self.dropout(torch.cat(pooled, dim = 1))

        #cat = [batch size, n_filters * len(filter_sizes)]
            
        return self.fc(cat)

In [0]:
'''FastText'''

import torch.nn as nn
import torch.nn.functional as F

class FastText(nn.Module):
    def __init__(self, vocab_size, embedding_dim, output_dim, pad_idx):
        
        super().__init__()
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=pad_idx)
        
        self.fc = nn.Linear(embedding_dim, output_dim)

        self
        
    def forward(self, text):
        
        #text = [sent len, batch size]
        
        embedded = self.embedding(text)
                
        #embedded = [sent len, batch size, emb dim]
        
        embedded = embedded.permute(1, 0, 2)
        
        #embedded = [batch size, sent len, emb dim]
        
        pooled = F.avg_pool2d(embedded, (embedded.shape[1], 1)).squeeze(1) 
        
        #pooled = [batch size, embedding_dim]
                
        

        return self.fc(pooled)


INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
OUTPUT_DIM = 8
DROPOUT = 0.5
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

model = FastText(INPUT_DIM, EMBEDDING_DIM, OUTPUT_DIM, PAD_IDX)

## 외부 임베딩 모델 통합
- 제공되는 임베딩 파일은 본 뉴스 기사를 word2vec으로 100차원으로 학습하여 text로 저장한 것.
- 이 파일을 불러들여 TorchTExt의 vocab과 vector에 연결할 필요
- gensim에서 제공하는 임베딩파일을 불러들이는 모듈을 사용할 수도 있고, 이 파일을 불러들이는 모둘을 직접 만들 수도 있다
- 형태소 분석 단위가 맞지 않아 없는 어휘들은 차원크기 만큼의 적당한 값으로 채울 필요 (0으로 채우든지 아니면 정규분포를 보이는 랜덤한 값으로 채우든지...)

In [302]:
# 제공된 NewsALlW2vec.txt를 통합하기 위한 코드
import gensim
import tqdm.notebook

#load_pretrained=gensim.models.KeyedVectors.load_word2vec_format('NewsAllW2vec.txt')
#pretrained_embeddings=torch.FloatTensor(load_pretrained.vectors)
w2v_model=gensim.models.KeyedVectors.load_word2vec_format('NewsAllW2vec.txt')
word2vec_vectors=[]
for token, idx in tqdm.notebook.tqdm(TEXT.vocab.stoi.items()):
    if token in w2v_model.wv.vocab.keys():
        word2vec_vectors.append(torch.FloatTensor(w2v_model[token]))
    else:
        word2vec_vectors.append(torch.zeros(100))
TEXT.vocab.set_vectors(TEXT.vocab.stoi, word2vec_vectors, 100)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:253: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  # Remove the CWD from sys.path while we load stuff.


In [303]:
#불러온 weight를 Embedding에 통합, 복사
pretrained_embeddings=torch.FloatTensor(TEXT.vocab.vectors)
print(pretrained_embeddings.shape)
model.embedding.weight.data.copy_(pretrained_embeddings)

torch.Size([20002, 100])


tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.5800, -1.5273,  2.1190,  ..., -2.8508, -3.2948, -0.9123],
        ...,
        [ 0.0247, -0.1477, -0.0120,  ...,  0.0290, -0.0606, -0.0663],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0195, -0.0469,  0.0379,  ...,  0.0415,  0.0296,  0.0534]])

In [0]:
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

In [305]:
# parameter 체크
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 2,001,008 trainable parameters


## Train and test the Model
- train, test accuracy 출력

In [0]:
import torch.optim as optim
#from torch.optim.lr_scheduler import StepLR

optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
#scheduler = StepLR(optimizer, step_size=1, gamma=0.1)

criterion = nn.CrossEntropyLoss()

model = model.to(device)
criterion = criterion.to(device)

In [0]:
def category_accuracy(preds, y):
    _, predicted=torch.max(preds.data,1)
    if torch.cuda.is_available():
      correct=(predicted.cpu()==y.cpu()).float()
    else:
      correct=(predicted==y).float()
    acc=correct.sum()/len(correct)
    return acc

In [0]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()

        predictions = model(batch.news).squeeze(1).to(device)
        #print(predictions.size())
        loss = criterion(predictions, batch.label.type(torch.LongTensor).to(device))
        
        acc = category_accuracy(predictions, batch.label)
        
        loss.backward()
        
        optimizer.step()

        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [0]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            predictions = model(batch.news).squeeze(1).to(device)
            loss = criterion(predictions, batch.label.type(torch.LongTensor).to(device))
            
            acc = category_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [0]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [311]:
N_EPOCHS = 30

best_test_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    test_loss, test_acc = evaluate(model, test_iterator, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if test_loss < best_test_loss:
        best_tes_loss = test_loss
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Test Loss: {test_loss:.3f} |  Test Acc: {test_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 0m 0s
	Train Loss: 1.872 | Train Acc: 40.47%
	 Test Loss: 1.705 |  Test Acc: 66.25%
Epoch: 02 | Epoch Time: 0m 0s
	Train Loss: 1.305 | Train Acc: 75.55%
	 Test Loss: 1.253 |  Test Acc: 72.19%
Epoch: 03 | Epoch Time: 0m 0s
	Train Loss: 0.749 | Train Acc: 87.03%
	 Test Loss: 0.917 |  Test Acc: 79.38%
Epoch: 04 | Epoch Time: 0m 0s
	Train Loss: 0.415 | Train Acc: 92.34%
	 Test Loss: 0.743 |  Test Acc: 80.94%
Epoch: 05 | Epoch Time: 0m 0s
	Train Loss: 0.236 | Train Acc: 96.25%
	 Test Loss: 0.656 |  Test Acc: 83.12%
Epoch: 06 | Epoch Time: 0m 0s
	Train Loss: 0.135 | Train Acc: 98.75%
	 Test Loss: 0.603 |  Test Acc: 84.69%
Epoch: 07 | Epoch Time: 0m 0s
	Train Loss: 0.079 | Train Acc: 99.53%
	 Test Loss: 0.577 |  Test Acc: 85.00%
Epoch: 08 | Epoch Time: 0m 0s
	Train Loss: 0.049 | Train Acc: 99.92%
	 Test Loss: 0.560 |  Test Acc: 84.69%
Epoch: 09 | Epoch Time: 0m 0s
	Train Loss: 0.034 | Train Acc: 100.00%
	 Test Loss: 0.547 |  Test Acc: 85.31%
Epoch: 10 | Epoch Time: 0m 

## USER Input : Prediction 모듈 설정
- 입력기사를 받아들여서 분류를 테스트 하는 모듈 작성

#### 네이버 뉴스 labels
    - 정치(0), 경제(1), 사회(2), 생활/문화(3), 세계(4), 기술/IT(5), 연예(6), 스포츠(7) 

In [0]:
def predict_news(model, sentence, min_len = 4):
    model.eval()
    tokenized = [tok for tok in tokenizer(sentence)]
    if len(tokenized) < min_len:
        tokenized += ['<pad>'] * (min_len - len(tokenized))
    indexed = [TEXT.vocab.stoi[t] for t in tokenized]
    tensor = torch.LongTensor(indexed).to(device)
    tensor = tensor.unsqueeze(1)
    preds = model(tensor)
    max_preds = preds.argmax(dim = 1)
    return max_preds.item()

In [313]:
## 다음 문장으로 각각 테스트
#- 정답은 6, 4, 0, 1, 2, 7, 5, 3

sentence = "화보 촬영 후 진행 된 인터뷰에서 처음으로 이름이 실시간 검색어에 올랐을 때의 소감을 묻자 처음엔 무서웠다. 내가 뭘 잘못했나 싶었다. 주변 사람들이 기뻐하는 것을 보며 긍정적으로 생각하게 됐다. 신원호 감독님은 숙명으로 받아 들이라고 하시더라라며 웃었다. 함께 출연한 배우들의 이야기도 빠질 수 없었다. 전미도는 다른 여성 출연자들과도 서로 얼굴 한 번 더 보려고 노력한다. 민하(안은진)와 겨울이(신현빈)와는 얼마 전에는 셋이 따로 처음으로 만나기도 했다. '99즈'와는 드라마가 끝난 후에도 한 달에 한 번은 만나서 합주를 한 상태다. 서로 정이 많이 들었다.라고 말하며 끈끈한 동료애를 드러냈다."#
#sentence = "아베 정권이 자의적으로 검찰 고위직 인사의 정년(停年)을 결정할 수 있게 하는 검찰청법 개정안에 대해 트위터를 중심으로 반대 여론이 집결됐다. 아사히 신문은 이 법안에 대해 리트윗을 포함한 반대가 700만건 이상 나타났다고 전했다. “코로나 사태로 어수선한 틈을 타서 독소조항이 들어있는 법안을 통과시키려 한다”는 여론이 순식간에 확산된 것이다. 입헌 민주당의 에다노 유키오(枝野幸男)대표는 아베 총리에게 “코로나 감염 확대 사태의 혼잡을 틈타서 불난 곳에서 도둑질 하려 한다”고 비판했다."
#sentence = "문재인 대통령은 18일 광주광역시 5·18민주광장에서 열린 제40주년 5·18민주화운동 기념식 기념사에서 “5·18 진상 규명에 최선을 다하겠다”며 “지난 12일 본격적으로 활동을 시작한 ‘5·18진상규명조사위원회’가 남겨진 진실을 낱낱이 밝힐 수 있도록 지원을 아끼지 않겠다”고 말했다."
#sentence = "이 같은 전망과 함께 구직자 중 76.9%는 ‘올해 안에 취업을 못할까 불안감을 느낀다’고 답했다. 불안감을 느끼는 주요 이유는 ‘경기불황으로 채용을 진행하는 기업 자체가 적을 것 같아서(71.7%)’, ‘상반기 채용 축소로 경쟁률이 더 높아질 것 같아서(50.0%)’, ‘기업들의 실적이 좋지 않다는 뉴스를 접해서(13.2%)’ 등이었다."
#sentence = "앞서 정부는 지난달 14일부터 재양성자가 지역 사회에서 또 다시 전파를 일으킬 것을 우려해 이들이 처음 확진 판정을 받았을 때처럼 병원 또는 집에 격리시켜왔다. 하지만 재양성자의 감염력이 없다는 판단에 따라 이들을 확진 환자에 준해 관리하지는 않겠다는 방침을 밝혔다. 윤태호 중앙사고수습본부 방역총괄반장은 “19일 0시부터 재양성자는 직장, 학교 등에 복귀한 경우 진단검사 결과 ‘음성’을 확인하지 않아도 되는 것으로 지침을 변경할 것”이라고 말했다."
#sentence = "손흥민은 EPL 통산 151경기서 51골을 기록했다. AFC는 “지난 4시즌 동안 모든 대회를 통틀어 75골을 넣으며 세계적인 명성을 얻었다”며 “지난 시즌엔 토트넘의 챔피언스리그 결승행에 핵심 역할을 했다”고 조명했다. EPL 통산 187경기서 15골을 기록한 기성용에 대해선 “200경기에 가까운 출전과 3개의 팀에서 정식선수로 뛰며 영국에서 멋진 커리어를 누렸다”며 “본업이 미드필더인 그는 2013년 잉글랜드 리그컵 결승서 중앙 수비수로 뛰며 구단 108년 역사상 유일한 메이저 트로피 획득을 도왔다”고 치켜세웠다."
#sentence = "LG전자는 최근 동영상 감상을 즐기는 사용자가 많아 Q61에 6.5인치 대화면 디스플레이를 탑재하고 전면 베젤도 최소화했다. 홀인(펀치홀) 디스플레이로 카메라가 화면에서 차지하는 면적도 최소화했다. 또 Q61에는 동영상 시청, 웹서핑 등 멀티미디어 환경에서 장시간 사용할 수 있도록 4000mAh(밀리암페어시)의 대용량 배터리가 들어갔다. LG전자는 “다양한 가격대와 디자인의 제품을 지속적으로 출시해 고객 선택 폭을 넓혀가겠다”고 밝혔다."
#sentence = "난데없이 삼성동이 외신(外信)의 집중 관심 지역으로 떠올랐다. 전광판 때문이다. 더 정확히는 케이팝광장 앞 코엑스 아티움 건물에 있는 대형 전광판에서 흘러나오는 미디어아트 ‘Wave’ 때문이다. 매시 정각과 30분마다 대략 1분 동안 시퍼런 파도가 요동치는 영상인데, 7K 고해상도로 펼쳐지는 ‘물쇼’를 투명 유리통 안에 가두는 연출을 통해 도심과의 접점을 극대화했다. 지난달 처음 설치돼 최근 유튜브 등을 통해 퍼져나가면서 특히 해외 소셜미디어가 들끓었고, 미국 포브스나 영국 데일리메일 등 유수의 매체도 앞다퉈 보도했다. CNN은 지난 20일 “거대한 입체 파도가 강남을 휩쓸었다”고 보도했다."
predict_news(model, sentence)

#2번째는 '정치'로, 8번째는 '세계'로 분류 된다.

6